<a href="https://colab.research.google.com/github/jae-gyeong/marchine_learning_start/blob/master/19_10_01tf_mnist_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 심층 신경망 구성

### 학습 내용
- MNIST 데이터 셋을 이용하여 심층 신경망을 구현한다.
- 은닉층이 2개인 신경망을 구현한다.


In [0]:
import tensorflow as tf

In [0]:
import os, warnings
# 경고 메시지 무시하거나 숨길때(ignore), 다시보이게(default)
# warnings.filterwarnings(action='default')
warnings.filterwarnings(action='ignore')


In [25]:
# 데이터 불러오기
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)


Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [26]:
# 데이터의 개수
print(mnist.test.num_examples, mnist.train.num_examples, mnist.validation.num_examples)
# 데이터의 행열 사이즈
print(mnist.train.labels.shape, mnist.train.images.shape)
print(mnist.test.labels.shape, mnist.test.images.shape)
print(mnist.validation.labels.shape, mnist.validation.images.shape)

10000 55000 5000
(55000, 10) (55000, 784)
(10000, 10) (10000, 784)
(5000, 10) (5000, 784)


## 01 신경망 모델 구성하기
- MNIST의 손글씨는 28 X 28로 구성되어 있다.
- 784개의 특징(픽셀)로 구성되어 있음.
- 레이블은 0~9까지의 10개의 분류

### 신경망 모델 구성하기
* 입력층 - 784 노드(unit)
* 은닉층(3개층) - 64노드(unit)
* 출력층(범주10개) - 10노드(unit)

In [0]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])


- tf.Variable(tf.random_normal([784, 256], stddev=0.01))
- 표준편차가 0.01인 정규 분포를 갖는 값.
- tf.matmul() 각 계층으로 들어오는 입력값에 각각의 가중치를 곱한다.
- tf.nn.relu() 활성화 함수로 ReLU를 사용하는 신경망 계층을 만든다.

In [59]:
W1 = tf.Variable(tf.random_normal([784,64], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_normal([64,64], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_normal([64,64], stddev=0.01))
L3 = tf.nn.relu(tf.matmul(L2, W3))

W4 = tf.Variable(tf.random_normal([64,10], stddev=0.01))
model = tf.nn.relu(tf.matmul(L3, W4))

print(W4)
print(model)


<tf.Variable 'Variable_25:0' shape=(64, 10) dtype=float32_ref>
Tensor("Relu_21:0", shape=(?, 10), dtype=float32)


## 비용함수, 최적화 함수 지정
#### AdamOptimizer (Adaptive Moment Estimation)은 RMSProp와 Momentum방식을 합친 것.


In [0]:
# old 버전 : cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost) #learning rate - 0.001


## 세션 생성 및 초기화

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

### 테스트 데이터와 학습 데이터를 분류하는 이유
- 과적합 해소

## 배치 사이즈 지정
- Mini-batch 크기가 전체 트레이닝 셋 데이터 사이즈인 m과 같다면 이것은 Batch gradient descent방법
   - 데이터가 별로 없다면 batch gradient descent를 사용
- Mini-batch 크기가 1이라면 Stochastic gradient descent라고 한다.
  - 적은 메모리로 동작가능
  - 64,128, 256, 512 사이즈 선택

In [63]:
# MNIST 데이터 전체를 학습하는 일을 15번 반복함.
# 학습 데이터 전체를 한 바퀴를 도는 일을 에포크(epoch)라 한다.
total_cost = 0


for i in range(55):
  # 배치 사이즈만큼 데이터 가져오기
  batch_xs, batch_ys = mnist.train.next_batch(1000) # 55000개를 1000개로 끊을 것
  # 입력값 : batch_xs, 출력값 : batch_ys
  # 최적화를 수행 후, 손실을 구한다.

  _, cost_val = sess.run([optimizer, cost], feed_dict={X:batch_xs, Y:batch_ys}) # 학습
  
 # 총loss 계산 - 55번 돌아가면서 발생된 cost 
  total_cost = total_cost + cost_val   # 한 epoch의 전체 평균을 구하기 위헤
  print("한번 학습시의 cost", cost_val )

print("1 Epoch 평균 cost : {:.3f}".format(total_cost/55))

한번 학습시의 cost 1.5988723
한번 학습시의 cost 1.536323
한번 학습시의 cost 1.5227116
한번 학습시의 cost 1.4990913
한번 학습시의 cost 1.5518794
한번 학습시의 cost 1.5308081
한번 학습시의 cost 1.4950247
한번 학습시의 cost 1.4738364
한번 학습시의 cost 1.5248457
한번 학습시의 cost 1.4922622
한번 학습시의 cost 1.4841309
한번 학습시의 cost 1.4782499
한번 학습시의 cost 1.4090122
한번 학습시의 cost 1.4331775
한번 학습시의 cost 1.4056135
한번 학습시의 cost 1.4321853
한번 학습시의 cost 1.3825121
한번 학습시의 cost 1.3447976
한번 학습시의 cost 1.376944
한번 학습시의 cost 1.3111184
한번 학습시의 cost 1.3282696
한번 학습시의 cost 1.2667658
한번 학습시의 cost 1.3374208
한번 학습시의 cost 1.305078
한번 학습시의 cost 1.285548
한번 학습시의 cost 1.2967273
한번 학습시의 cost 1.2904038
한번 학습시의 cost 1.2693616
한번 학습시의 cost 1.2918152
한번 학습시의 cost 1.2991042
한번 학습시의 cost 1.2574854
한번 학습시의 cost 1.2598054
한번 학습시의 cost 1.1786697
한번 학습시의 cost 1.2281739
한번 학습시의 cost 1.234719
한번 학습시의 cost 1.1838328
한번 학습시의 cost 1.219078
한번 학습시의 cost 1.2055799
한번 학습시의 cost 1.1948063
한번 학습시의 cost 1.1894933
한번 학습시의 cost 1.2377994
한번 학습시의 cost 1.1805483
한번 학습시의 cost 1.1567215
한번 학습시의 cost 1.22

In [0]:
# batch_zixe 100 에폭 30 
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [80]:
for epoch in range(30):
  total_cost = 0
  
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(100) # 55000개를 100개로 끊을 것
    _, cost_val = sess.run([optimizer, cost], feed_dict={X:batch_xs, Y:batch_ys}) # 학습

   # 총loss 계산 - 55번 돌아가면서 발생된 cost 
    total_cost = total_cost + cost_val   # 한 epoch의 전체 평균을 구하기 위헤
  print("Epoch : %4d" %(epoch + 1), '평균 Cost = ', "{:.3f}".format(total_cost/100))
 # print("1 Epoch 평균 cost : {:.3f}".format(total_cost/total_batch))

Epoch :    1 평균 Cost =  1.423
Epoch :    2 평균 Cost =  1.437
Epoch :    3 평균 Cost =  1.436
Epoch :    4 평균 Cost =  1.426
Epoch :    5 평균 Cost =  1.436
Epoch :    6 평균 Cost =  1.416
Epoch :    7 평균 Cost =  1.576
Epoch :    8 평균 Cost =  1.461
Epoch :    9 평균 Cost =  1.458
Epoch :   10 평균 Cost =  1.435
Epoch :   11 평균 Cost =  1.437
Epoch :   12 평균 Cost =  1.486
Epoch :   13 평균 Cost =  1.433
Epoch :   14 평균 Cost =  1.478
Epoch :   15 평균 Cost =  1.446
Epoch :   16 평균 Cost =  1.432
Epoch :   17 평균 Cost =  1.437
Epoch :   18 평균 Cost =  1.426
Epoch :   19 평균 Cost =  1.475
Epoch :   20 평균 Cost =  1.483
Epoch :   21 평균 Cost =  1.448
Epoch :   22 평균 Cost =  1.452
Epoch :   23 평균 Cost =  1.445
Epoch :   24 평균 Cost =  1.452
Epoch :   25 평균 Cost =  1.438
Epoch :   26 평균 Cost =  1.465
Epoch :   27 평균 Cost =  1.447
Epoch :   28 평균 Cost =  1.449
Epoch :   29 평균 Cost =  1.451
Epoch :   30 평균 Cost =  1.452
